# Table of Contents
1. Import libraries
2. Import data: ords_prods_merge2_extrarows.pkl
3. Misc task work
4. Analyzing spending habit differences between customer loyalty segments
5. Data cleaning to resolve outlier prices
6. Back to analyzing spending habit differences between customer loyalty segments
7. Create a 'product_price spending flag' for each user based on the avg product price across all their orders
8. Create a 'avg_order_spending_flag' for each user based on the avg order price across all their orders
9. Create 'frequency_flag' based on 'median_days_since_prior_order
10. Export data: ords_prods_merge4.pkl

# Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os

# Import Data

In [2]:
# define path
path = r'/users/stephenhelvig/documents/python projects/instacart basket analysis'

In [3]:
# import ords_prods_merge2
df = pd.read_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge2_extrarows.pkl'))

# Task Work

In [4]:
# Find the aggregate mean of the 'order_number' column grouped by 'department_id'

df.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457687
2,17.277920
3,17.179756
4,17.811403
5,15.215751
6,16.439806
7,17.225773
8,15.340520


The results of the order_number mean are very similar to the subset results. I don't think any department's mean changed more than 1. 

The loyalty flag was created for this dataset during the exercise

In [5]:
df['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15891172
Loyal customer      10293959
New customer         6249358
Name: count, dtype: int64

# Analyzing spending habit differences between customer loyalty segments

In [2]:
# The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers 
# you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category 
# (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by 
# loyal customers differ from those purchased by regular or new customers.

In [6]:
df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.389099  1.0  99999.0
New customer      13.294927  1.0  99999.0
Regular customer  12.496699  1.0  99999.0

In [3]:
# This outlier 99999 is not helpful. I'm going to do some additional cleaning.

# Data cleaning to resolve outlier prices

In [7]:
# Look at all products with a price >= 1000
df_high_prices = df[df['prices'] >= 1000]

In [8]:
df_high_prices

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,max_order,loyalty_flag
1576,912404,17,12,2,14,5.0,False,21553,5,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,40,Regular customer
1638,603376,17,22,6,16,4.0,False,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,40,Regular customer
16534,3264360,135,2,2,21,13.0,False,21553,6,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,4,New customer
16540,892534,135,3,0,8,12.0,False,21553,3,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,4,New customer
53711,229704,342,8,1,19,30.0,False,21553,9,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,16,Regular customer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32350064,3172853,205650,18,1,9,7.0,False,21553,17,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,25,Regular customer
32377297,2504315,205818,3,5,15,3.0,False,21553,13,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,25,Regular customer
32377307,1108388,205818,5,4,5,1.0,False,21553,5,1,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,25,Regular customer
32410137,1916142,206049,1,2,17,NaN,True,21553,2,0,Lowfat 2% Milkfat Cottage Cheese,108,16,14900.0,5,New customer


In [9]:
# Get unique product ids, names, & prices for rows >= 1000
unique_high_product_id = df_high_prices['product_id'].unique()
unique_high_product_name = df_high_prices['product_name'].unique()
unique_high_product_prices = df_high_prices['prices'].unique()
print(unique_high_product_id)
print(unique_high_product_name)
print(unique_high_product_prices)
print("Number of unique product ids:", len(unique_high_product_id))

[21553 33664]
['Lowfat 2% Milkfat Cottage Cheese' '2 % Reduced Fat  Milk']
[14900. 99999.]
Number of unique product ids: 2


In [10]:
# changing the price of Lowfat 2% Milkfat Cottage Cheese from 14900.0 to 4.99
df.loc[df['prices'] == 14900, 'prices'] = 4.99

In [11]:
# changing the price of 2% Reduced Fat Milk from 99999 to 3.49
df.loc[df['prices'] == 99999, 'prices'] = 3.49

In [12]:
# checking output
df.loc[df['product_id'].isin([21553, 33664]), ['product_id', 'product_name', 'prices']] \
    .drop_duplicates()

,product_id,product_name,prices
1576,21553,Lowfat 2% Milkfat Cottage Cheese,4.99
129036,33664,2 % Reduced Fat Milk,3.49


# Back to analyzing spending habit differences between customer loyalty segments

In [13]:
# Looking at spending habits:

df.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices           
                      mean  min   max
loyalty_flag                         
Loyal customer    7.778470  1.0  25.0
New customer      7.805616  1.0  25.0
Regular customer  7.802993  1.0  25.0

Very little difference between the customer groups

# Create a 'product_price spending flag' for each user based on the avg product price across all their orders

In [14]:
# Aggregating data with transform()
df['users_avg_product_price'] = df.groupby(['user_id'])['prices'].transform('mean')

In [15]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,max_order,loyalty_flag,users_avg_product_price
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,10,New customer,6.367797
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,10,New customer,6.367797
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,10,New customer,6.367797
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,10,New customer,6.367797
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,10,New customer,6.367797


In [16]:
# Assigning a product_price_spending_flag based on the users_avg_product_price. 

df.loc[df['users_avg_product_price'] >= 10, 'product_price_spending_flag'] = 'High spender'
df.loc[df['users_avg_product_price'] < 10,  'product_price_spending_flag'] = 'Low spender'

In [17]:
# checking output
df['product_price_spending_flag'].value_counts(dropna = False)

product_price_spending_flag
Low spender     32313719
High spender      120770
Name: count, dtype: int64

In [18]:
# checking output
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,max_order,loyalty_flag,users_avg_product_price,product_price_spending_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,10,New customer,6.367797,Low spender
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,10,New customer,6.367797,Low spender
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,10,New customer,6.367797,Low spender
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,10,New customer,6.367797,Low spender
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,10,New customer,6.367797,Low spender


# Create a 'avg_order_spending_flag' for each user based on the avg order price across all their orders

In [19]:
# Aggregating order totals data with transform:
df['order_total'] = df.groupby(['user_id', 'order_number'])['prices'].transform('sum')

In [20]:
# Collapse to one row per order to avoid double-counting
order_totals = (df.groupby(['user_id', 'order_number'])['prices'].sum())

In [21]:
# Average spend per user (mean of their order totals)
avg_spend_per_user = order_totals.groupby('user_id').mean()

In [22]:
# Map that back to the row-level dataframe
df['avg_spend_per_user'] = df['user_id'].map(avg_spend_per_user)

In [23]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,max_order,loyalty_flag,users_avg_product_price,product_price_spending_flag,order_total,avg_spend_per_user
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,10,New customer,6.367797,Low spender,31.6,37.57
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,10,New customer,6.367797,Low spender,31.6,37.57
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,10,New customer,6.367797,Low spender,31.6,37.57
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,10,New customer,6.367797,Low spender,31.6,37.57
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,10,New customer,6.367797,Low spender,31.6,37.57


In [24]:
# find the mean of avg_spend_per_user
df['avg_spend_per_user'].describe()

count    3.243449e+07
mean     1.028080e+02
std      5.118860e+01
min      1.000000e+00
25%      6.649322e+01
50%      9.407273e+01
75%      1.297167e+02
max      5.512200e+02
Name: avg_spend_per_user, dtype: float64

In [25]:
# Assigning a new flag based on the avg_spend_per_user
df.loc[df['avg_spend_per_user'] >= 130, 'avg_order_spending_flag'] = 'High order spender'
df.loc[df['avg_spend_per_user'] < 66, 'avg_order_spending_flag'] = 'Low order spender'
df.loc[(df['avg_spend_per_user'] >= 66) & (df['avg_spend_per_user'] < 130), 'avg_order_spending_flag'] = 'Avg order spender'

In [26]:
df['avg_order_spending_flag'].value_counts(dropna = False)

avg_order_spending_flag
Avg order spender     16406233
High order spender     8057349
Low order spender      7970907
Name: count, dtype: int64

In [27]:
# Task 7: Create an order frequency flag according to the median in 'days_since_prior_order' column. 

# Aggregating data with transform()

df['median_days_since_prior_order'] = df.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [28]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,department_id,prices,max_order,loyalty_flag,users_avg_product_price,product_price_spending_flag,order_total,avg_spend_per_user,avg_order_spending_flag,median_days_since_prior_order
0,2539329,1,1,2,8,NaN,True,196,1,0,...,7,9.0,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,16,12.5,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,19,4.4,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,19,4.7,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,17,1.0,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5


# Create 'frequency_flag' based on 'median_days_since_prior_order

In [29]:
# assigning a frequency flag based on median_days_since_prior_order value

df.loc[df['median_days_since_prior_order'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [30]:
df.loc[df['median_days_since_prior_order'] <= 20, 'frequency_flag'] = 'Frequent customer'

In [31]:
# checking outputs
df['frequency_flag'].value_counts(dropna = False)

frequency_flag
Frequent customer        28794843
Non-frequent customer     3639646
Name: count, dtype: int64

In [32]:
df.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,...,prices,max_order,loyalty_flag,users_avg_product_price,product_price_spending_flag,order_total,avg_spend_per_user,avg_order_spending_flag,median_days_since_prior_order,frequency_flag
0,2539329,1,1,2,8,NaN,True,196,1,0,...,9.0,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5,Non-frequent customer
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,12.5,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5,Non-frequent customer
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,4.4,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5,Non-frequent customer
3,2539329,1,1,2,8,NaN,True,26088,4,0,...,4.7,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5,Non-frequent customer
4,2539329,1,1,2,8,NaN,True,26405,5,0,...,1.0,10,New customer,6.367797,Low spender,31.6,37.57,Low order spender,20.5,Non-frequent customer


# Export Data

In [33]:
df.to_pickle(os.path.join(path, 'Data', 'Prepared Data', 'ords_prods_merge4.pkl'))